# Diamond Price porject
Diamonds are considered very precious in our society and are also highly valued monetarily. In this project, let's create a model which predicts the price of a diamond based on its features.

## 1. Problem
Since we have to predict the price of the diamond, which is a number. Hence this is a regression problem.

## 2. Data
The data taken for this project is taken from Kaggle. It contains various features of diamonds and its price. The description of its features are:
    
    1. carat weight of the diamond (0.2--5.01)
    2. cut quality of the cut (Fair, Good, Very Good, Premium, Ideal)
    3. color diamond colour, from J (worst) to D (best)
    4. clarity a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
    5. x length in mm (0--10.74)
    6. y width in mm (0--58.9)
    7. z depth in mm (0--31.8)
    8. depth total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)
    9. table width of top of diamond relative to widest point (43--95)
    
The predicting variable is :
    
    price in US dollars (\$326--\$18,823)
    
Link to the dataset : https://www.kaggle.com/shivam2503/diamonds/download

## 3. Evaluation
Since it is a regression model, we will evaluate it on metrics like r2 score, mean absolute error and mean squared error.

## 4. Data Analysis

Let's analyse the data by running some visualization on it.

In [ ]:
# Importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Importing project csv file
df = pd.read_csv("../input/diamonds/diamonds.csv")
df.head()

This column named 'Unnamed: 0' is kind of useless. So it will fine if it is deleted.

In [ ]:
df.drop('Unnamed: 0',axis = 1,inplace = True)

In [ ]:
df.head()

In [ ]:
# Information about the data set
df.info()

There are a lot of samples in this data set, hence it will be fun running visualizations on it.

### Running visulizations on numerical data
Let's compare numerical features against the price column. Since there are less features, we can easily cover most of the correlations present in the data.

In [ ]:
# Distribution of carat values in diamonds
df["carat"].plot.hist(color = "lightblue");

In [ ]:
# Relation b/w carat and price 
fig,ax = plt.subplots(tight_layout = True,figsize = (10,10))
ax.scatter(df['carat'],df['price'],color = 'salmon')
ax.set(title = "Relation b/w carat and price of diamond",
       xlabel = "Carat",
       ylabel = "Price(in USD)");

This graph shows that there is a heavy correlation b/w carat and price, diamonds with large values of carat have high prices.

In [ ]:
# Distribution of depth
df['depth'].plot.hist();

In [ ]:
# Relation b/w price and depth 
fig,ax = plt.subplots(tight_layout = True,figsize = (10,10))
ax.scatter(df['depth'],df['price'],color = 'lightgreen')
ax.set(title = "Relation b/w price and depth of diamond",
       xlabel = "Depth (z/mean(x,y))",
       ylabel = "Price (in USD)");

This plot shows that most diamonds have depth ratio in the range of 50-75. There are some outliers but they do not show any significant effect.

In [ ]:
# Distribution of table
df['table'].plot.hist(color = 'indigo');

In [ ]:
# Relation b/w price and table
fig,ax = plt.subplots(tight_layout = True,figsize = (10,10))
ax.scatter(df['table'],df['price'],color = 'pink')
ax.set(title = 'Relation b/w table and price',
       xlabel = 'Table',
       ylabel = 'Price (in USD)');

This feature also has same properties as in depth feature. The range here is 50-70. Here also the outliers also don't show any significance

In [ ]:
# Relation b/w dimensions and price
dim = ['x','y','z']
colors = ['r','b','g']
fig,ax = plt.subplots(1,3,tight_layout = True,figsize = (15,10))
for i in range(3):
    ax[i].scatter(df[dim[i]],df['price'],color = colors[i])
ax[0].set_xlabel('X')
ax[1].set_xlabel('Y')
ax[2].set_xlabel('Z');

1. The X feature has high correlation, although there are some outliers.
2. The Y and Z feature has way more correlation and it has less outliers. Also, their values is less than that of X feature.

We will remove these outliers for:
 
 1. X in range 0-3.
 2. Y in range >20.
 3. Z in range >15.

In [ ]:
# Relation b/w price and carat w.r.t table
fig,ax = plt.subplots(tight_layout =True,figsize = (10,10))
ax.scatter(df['carat'],df['price'],c = df['table'],cmap = 'Blues')
ax.set(title = "Relation b/w price and carat w.r.t table",
       xlabel = 'Carat',
       ylabel = 'Price(in USD)');

This shows that table value doesn't have much significance regarding carat and price. Regardless of carat values, the diamonds have similar table value. 

In [ ]:
# Relation b/w price and carat w.r.t depth
fig,ax = plt.subplots(tight_layout = True,figsize = (10,10))
ax.scatter(df['carat'],df['price'],c = df['depth'],cmap = 'plasma')
ax.set(title = "Relation b/w price and carat w.r.t. depth",
       xlabel = 'Carat',
       ylabel = 'Price(in USD)');

The depth feature also shows same behaviour as shown by table feature, as most diamonds have same values of depth regardless of carat

In [ ]:
# Relation of carat with table and depth
fig,ax = plt.subplots(1,2,tight_layout = True,figsize = (20,10))
ax[0].scatter(df['carat'],df['table'],color = 'lightblue')
ax[0].set(title = "Relation b/w table and carat",
          xlabel = "Carat",
          ylabel = 'Table')
ax[1].scatter(df['carat'],df['depth'],color = 'purple')
ax[1].set(title = "Relation b/w depth and carat",
              xlabel = "Carat",
              ylabel = "Depth");

This proves that depth and carat value remains in a specific range.

In [ ]:
# Relation b/w carat and dimensions w.r.t price
fig,ax = plt.subplots(1,3,tight_layout = True,figsize = (15,10))
for i in range(3):
    ax[i].scatter(df[dim[i]],df['carat'],c = df['price'],cmap = 'viridis')
    ax[i].set(xlabel = dim[i],
              ylabel = 'Carat');

These plots show that :
    
    1. All dimension variables have almost linear relation with carat, with varying slope.
    2. Price starts increasing after 1 carat.

### Visualizations on categorical data
Let's run some visualization of categorical data present with us to see its impact on price.

In [ ]:
pip install seaborn

Seaborn is more suitable in plotting categorical data. So let's use it to make some visualizations.

In [ ]:
import seaborn as sns

In [ ]:
# Bar graphs for categorical features
categorical_features = ['cut','color','clarity']
fig,ax = plt.subplots(1,3,tight_layout = True,figsize = (15,10))
for i in range(3):
    df[categorical_features[i]].value_counts().T.plot.bar(color = colors[i],ax = ax[i])
    ax[i].set_title(categorical_features[i]);

Points :
    
    1. Most diamonds have ideal cut.
    2. Most diamonds have G colour.
    3. Most diamonds have S1 colour.

In [ ]:
# Comparing price of diamond in different categories
sns.catplot(x = 'cut',y = 'price',data = df,height = 9,aspect = 11.7/8.27,ax = ax[i]);

In [ ]:
sns.catplot(x = 'color',y = 'price',data = df,height = 9,aspect = 11.7/8.27,ax = ax[i]);

In [ ]:
sns.catplot(x = 'clarity',y = 'price',data = df,height = 9,aspect = 11.7/8.27,ax = ax[i]);

Points to be noted : 

    1. In 'cut' category, the categories 'ideal','premium' and 'very good' have diamonds in every price range, while the other categories have very few samples in higher price range.
    2. In 'color' category, all categories have samples in every price range
    3. In 'clarity' category, except 'I1', 'WS2','WS1 and 'IF' posses the same property.
    4. These plots give only a hint at their correlation with price.

Let's compare one category w.r.t. another and see what kind of results we get.

In [ ]:
# Relation b/w price and cut w.r.t colour
sns.catplot(x = 'cut',y = 'price',data = df,height = 11,aspect = 11.7/8.27,hue = 'color');

Points to be noted:
    
    1. Colours 'I','G' and 'H' are found in each 'cut'.
    2. Colour 'E' and 'J' are not prevalent in higher price range.
    3. Colour 'D' is found in low to mid range.
    4. All cuts have all colours in them.

In [ ]:
# Relation b/w price and clarity w.r.t. colour
sns.catplot(x = 'cut',y = 'price',hue = 'clarity',data = df,height = 11, aspect = 11.7/8.27);

Points to be noted:
    
    1. The clarity categories 'S1','S2','VS1' and 'VS2' are present in all cuts.
    2. The rest are present in low to mid range.
    3. All cut have all types of clarity in them.

In [ ]:
# Relation b/w price and clarity w.r.t. colour
sns.catplot(x = 'clarity',y = 'price', hue = 'color',data = df,height = 11, aspect = 11.7/8.27);

Points :
    
    1. In 'clarity' types 'SI1','SI2','VS1' and 'VS2' have colour 'I' dominating in the higher price range.
    2. The colour 'G' is mid range in the above mentioned types, but is high range in 'WS1' amd 'WS2'.
    3. The colour 'F' is present in all price ranges and clarity types.
    4. The colour 'H' is high range in 'SI1', 'SI2' and 'VS2'.

In [ ]:
# Plotting price and carat w.r.t. categories ('IF','I1','VVS1','VVS2')
fig,ax = plt.subplots(figsize = (10,10),tight_layout = True)
ax.scatter(df.carat[df.clarity == 'IF'],df.price[df.clarity == 'IF'],color = 'salmon')
ax.scatter(df.carat[df.clarity == 'I1'],df.price[df.clarity == 'I1'],color = 'lightblue')
ax.scatter(df.carat[df.clarity == 'VVS1'],df.price[df.clarity == 'VVS1'],color = 'lightgreen')
ax.scatter(df.carat[df.clarity == 'VVS2'],df.price[df.clarity == 'VVS2'],color = 'orange');

In [ ]:
# Plotting for remaining categories of clarity
fig,ax = plt.subplots(1,2,figsize = (10,10),tight_layout = True)
cl1 = ['SI1','SI2','VS1','VS2']
clr = ['salmon','lightblue','lightgreen','orange']
for i in range(4):
    if i < 2:
        ax[0].scatter(df.carat[df['clarity']==cl1[i]],df.price[df['clarity']==cl1[i]],color = clr[i])
        ax[0].set_xticks(range(6))
    else:
        ax[1].scatter(df.carat[df['clarity']==cl1[i]],df.price[df['clarity']==cl1[i]],color = clr[i])
        ax[1].set_xticks(range(6));

These plots show that these 4 categories are having high correlation with price.

Points : 
    
    1. The categories 'IF','I1' and 'VVS2' have high impact on price of diamond, as the slope is greater.
    2. The category 'VVS1' has almost a linear relation with price.
    3. The categories other than that mentioned in above points have higher range of price.
    
Hence, the clarity categories in point 1 are indicators of higher price.

In [ ]:
# Relation of price and carat w.r.t cut types ('Ideal','Premium','Very Good') and ('Good cut','Fair') in separate plots
fig,ax = plt.subplots(1,2,figsize = (15,12),tight_layout = True)
ct1 = df['cut'].unique()
clr = ['salmon','lightblue','lightgreen','orange','indigo']
print(ct1)
for i in range(5):
    if i < 3:
        ax[0].scatter(df.carat[df['cut']==ct1[i]],df.price[df['cut']==ct1[i]],color = clr[i])
        ax[0].set_xticks(range(6))
    else:
        ax[1].scatter(df.carat[df['cut']==ct1[i]],df.price[df['cut']==ct1[i]],color = clr[i])
        ax[1].set_xticks(range(6));

Clearly we can see that cuts ('Ideal','Premium','Very Good') have higher price for lower carats. Hence, such diamonds will be more expensive.

In [ ]:
# Relation b/w carat and price w.r.t. different colors
fig,(ax0,ax1) = plt.subplots(2,4,tight_layout = True,figsize = (12,12))
print(df.color.unique())
clr = ['salmon','lightblue','lightgreen','orange','indigo','yellow','pink']
for i in range(7):
    if i < 4:
        ax0[i].scatter(df.carat[df.color == df.color.unique()[i]],df.price[df.color == df.color.unique()[i]],color = clr[i])
        ax0[i].set_xticks(range(6))
    else:
        ax1[i-4].scatter(df.carat[df.color == df.color.unique()[i]],df.price[df.color == df.color.unique()[i]],color = clr[i])
        ax1[i-4].set_xticks(range(6));

These plots show that for all colours are showing almost same correlation b/w price and carat. Hence we can see that there are same patterns of prices for different colours.

### Conclusion
The information obtained about the data after all visualizations are : 

    1. Carat has high correlation with price, so does the dimension.
    2. Depth and table have values in the same range for all carats and prices, hence have relatively low correlation.
    3. Depth and table are dependent on dimensions as mentioned earlier in description, and they have values in specific ranges, hence they do not impact dimensions.
    4. The clarity 'VVS2' has lower correlation to price than other clarity types.
    5. The 'Fair' has lower price than other cuts.
    6. The price and color correlation is almost the same for all color types.

At last, let's make a correlation matrix to check whether our observations are correct or not(only for numeric data).

In [ ]:
# Drawing correlation matrix
corr = df.corr()
corr.style.background_gradient(cmap = "winter").set_precision(2)

The correlation matrix show : 
    
    1. The dimensions and carat values have high correlation with price(almost linear).
    2. Depth and table have low correlation with every other feature.
    
Hence, the data analysis was quite on point.

## 5. Modelling 
Since we have a lot samples in our dataset, we must use a small yet significant portion of the data to see the training results, as large number of samples take a lot of time to train. For training, let's take 10% of total samples. This much sample will ensure that the model trains on enough data and it will also save our time.

In [ ]:
# Taking 10% random samples from main dataset for training 
np.random.seed(42)
train = df.sample(frac = 0.1)
train.head()

In [ ]:
# Some information about the subset
train.info()

We now have 5394 samples, hence it would be easy and time saving for us to train the model. Also, the rows are selected randomly, so the patterns in the original dataset can be easily found here.

In [ ]:
# Splitting into features and label
X = train.drop('price',axis = 1)
y = train['price']

In [ ]:
X.head()

In [ ]:
y.head()

Since we have categorical features, we need to convert them into numeric features. Let's use one hot encoding to do this.

In [ ]:
# One hot encoding data
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
column = make_column_transformer((OneHotEncoder(drop = 'if_binary',sparse = False),['cut','color','clarity']),remainder = 'passthrough')
column.fit_transform(X)

In [ ]:
X.head()

Now, we have made column transformer, let's now split the data into training and test set.

In [ ]:
# Splitting data into training and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

The data is now split into training and test set. Let's start modelling.

Since it is a regression problem, hence we will start from Ridge regression model.

In [ ]:
# Making pipeline function to model using different models and evaluating them
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import matplotlib.ticker as ticker
def pipeline(model,X_train,y_train,X_test,y_test):
    pipe = make_pipeline(column,model)
    pipe.fit(X_train,y_train)
    # Evaluating metrics on training and test set
    train_predict = pipe.predict(X_train)
    test_predict = pipe.predict(X_test)
    scores = {
        "Training R^2 score" : r2_score(y_train,train_predict),
        "Test R^2 score" : r2_score(y_test,test_predict),
        "Training MAE" : mean_absolute_error(y_train,train_predict),
        "Test MAE" : mean_absolute_error(y_train,train_predict),
        "Training MSE" : mean_squared_error(y_train,train_predict),
        "Test MSE" : mean_squared_error(y_test,test_predict)
    }
    print(scores)
    fig,ax = plt.subplots(1,3,tight_layout = True,figsize = (9,9))
    ax[0].bar(['Training','Test'],[r2_score(y_train,train_predict),r2_score(y_test,test_predict)],color = 'lightblue')
    ax[0].set_title('R^2 score')
    ax[1].bar(['Training','Test'],[mean_absolute_error(y_train,train_predict),mean_absolute_error(y_train,train_predict)],color = 'lightblue')
    ax[1].set_title('MAE')
    ax[2].bar(['Training','Test'],[mean_squared_error(y_train,train_predict),mean_squared_error(y_test,test_predict)],color = 'lightblue');
    ax[2].set_title('MSE');
    return pipe

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
pipeline(ridge,X_train,y_train,X_test,y_test)

Although the R^2 score is coming high, the MAE and MSE values are coming very large, which is due to large difference in maximum and minimum values. Let's delete some features and run the model again.

We have seen that the 'depth' and 'table' values have very less significance, let's remove them and train the model again. 

In [ ]:
# Removing 'depth' and 'table' from main dataset
X1 =X.drop(['depth','table'],axis = 1)
X1.head()

In [ ]:
# Training model on modified data
np.random.seed(42)
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y,test_size = 0.2)
pipeline(ridge,X1_train,y1_train,X1_test,y1_test)

In genral, training metrics have worsened but test metrics have really improved. Let's see what features we can modify and see what results we can see.

In [ ]:
# Dropping some potential outliers
X2 = X.drop(X[(X.x<3) & (X.y>20) & (X.z >15)].index)
np.random.seed(42)
X2_train,X2_test,y2_train,y2_test = train_test_split(X2,y,test_size = 0.2)
pipeline(ridge,X2_train,y2_train,X2_test,y2_test);

The model seems to reach its maximum performance. Let's try another model and see the results.

In [ ]:
# Using LinearRegression model
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
pipeline(linear,X_train,y_train,X_test,y_test)

The LinearRegression() model is performing better on training set and worse in test set, let's train it on the data not having 'depth' and 'table' features.

In [ ]:
pipeline(linear,X1_train,y1_train,X1_test,y1_test)

The training metrics have very small change, but the test metrics have improved.

Let's train the model on the data which doesn't have some outliers.

In [ ]:
pipeline(linear,X2_train,y2_train,X2_test,y2_test)

Let's modify some more features and see what results are in front of us.

Earlier we saw that the 'Fair' cut diamonds have lower price to carat relation. Let's remove that feature and see what happens.

In [ ]:
train1 = train
train1.drop(train1[train1.cut == 'Fair'].index,inplace = True)
np.random.seed(42)
X3 = train1.drop('price',axis = 1)
y3 = train1['price']
X3_train,X3_test,y3_train,y3_test = train_test_split(X3,y3,test_size = 0.2)
pipeline(linear,X3_train,y3_train,X3_test,y3_test)

Except MSE on test set, all other metrics have improved significantly.

In [ ]:
# Training with Ridge() model
pipeline(ridge,X3_train,y3_train,X3_test,y3_test)

In the Ridge() model, R^2 score on both sets remain somewhat the same, with MAE and MSE on both sets worsening.

In [ ]:
# Let's remove 'color' columns, as we saw all colours have same correlation with price and carat
train2 = train1
train2.drop('color',axis = 1)
X4 = train2.drop('price',axis = 1)
y4 = train2['price']
np.random.seed(42)
X4_train,X4_test,y4_train,y4_test = train_test_split(X4,y4,test_size = 0.2)
pipeline(ridge,X4_train,y4_train,X4_test,y4_test)

In this case also, R^2 score remains the same, but the MAE and MSE are worsening.

In [ ]:
# Training on LinearRegression() model
pipeline(linear,X4_train,y4_train,X4_test,y4_test)

The LinaerRegression() model performs a bit better than the Ridge model, but still it is worse than that in previous cases.

Let's remove the clarity 'VVS2' as it was seen to have low correlation.

In [ ]:
train3 = train2
train3.drop(train3[train3.clarity == 'VVS2'].index,inplace = True)
X5 = train3.drop('price',axis = 1)
y5 = train3.price
np.random.seed(42)
X5_train,X5_test,y5_train,y5_test = train_test_split(X5,y5,test_size = 0.2)
pipeline(ridge,X5_train,y5_train,X5_test,y5_test)

The R^2 score remains somewhat the same, MAE has worsened and MSE has improved significantly.

In [ ]:
# Using LinearRegression() model
pipeline(linear,X5_train,y5_train,X5_test,y5_test)

This model's R^2 improved a bit, MAE and MSE has improved from previous case.

Let's use a different model and see what results it shows.

In [ ]:
# Using RandomForestRegressor() model
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
pipeline(forest,X_train,y_train,X_test,y_test)

The metrics have significantly improved from the previous two models. Let's see how it performs on modified data


In [ ]:
# Using RandomForestRegressor() on data not having 'depth' and 'table'
pipeline(forest,X1_train,y1_train,X1_test,y1_test)

The metrics are keeping on improving, although the metrics may seem to show overfitting. Let's train it on more data.

In [ ]:
# Training on data not having 'Fair' cut 
pipeline(forest,X3_train,y3_train,X3_test,y3_test)

Except R^2, other metrics are worsening a bit. Let's try it another set of modified data

In [ ]:
# Using data without color columns
pipeline(forest,X4_train,y4_train,X4_test,y4_test)

There has been a improvement in the metrics, but it is very small.

In [ ]:
# Using data not having 'VVS2' clarity
pipeline(forest,X5_train,y5_train,X5_test,y5_test)

The metrics have started worsening, hence based on metrics,we should try the RandomForestRegressor() model with the data not having 'depth','table','Fair' cut and 'VVS2' clarity.

In [ ]:
# Doing final modfications on data
train4 = train
train4.drop(['depth','table'],axis = 1)
train4.drop(train4[train4.cut == 'Fair'].index,inplace = True)
train4.drop(train4[train4.clarity == 'VVS2'].index,inplace = True)
X6 = train4.drop('price',axis = 1)
y6 = train4.price
np.random.seed(42)
X6_train,X6_test,y6_train,y6_test = train_test_split(X6,y6,test_size = 0.2)
pipeline(forest,X6_train,y6_train,X6_test,y6_test)

After all data munging, the model seems to not improve from the result we have seen after removing 'depth' and 'table' feature. Hence, we should stick with doing only that change in the final dataset.

In [ ]:
train.head()

In [ ]:
# Cross validation final selected model
train.drop(['depth','table'],axis = 1,inplace = True)
X = train.drop('price',axis  = 1)
y = train.price
pipe = make_pipeline(column,forest)
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X,y).mean()

Even in cross validation, the model seems to perfrom a lot better than expected. This may be possible due to:
    
    1. Large number of samples to train on.
    2. Removing features which have very low correlation.
    
Let's see the cross validation score for MAE and MSE.

In [ ]:
# Cross validation for MAE and MSE
print(abs(cross_val_score(pipe,X,y,scoring = 'neg_mean_absolute_error').mean()))
print(abs(cross_val_score(pipe,X,y,scoring = 'neg_mean_squared_error').mean()))

The results for MAE and MSE are worse than expected, but still better than what was seen in initial models.

### Final Modelling

Let's do modelling on now the full data. Here, we will now make three different sets, train, test and valid. Valid set will be used for cross validation, while the other two will be used for the same purpose they were used earlier.

In [ ]:
# Making changes in original data
df.drop(['depth','table'],axis = 1,inplace = True)
df = df.sample(frac = 1)
train_set = df[:round(0.6*len(df))]
valid_set = df[round(0.6*len(df)):round(0.8*len(df))]
test_set = df[round(0.8*len(df)):]

In [ ]:
# Shapes of different sets
train_set.shape,valid_set.shape,test_set.shape

Now, we have made the final split, let's start doing the final modelling process.

In [ ]:
X_train = train_set.drop('price',axis = 1)
y_train = train_set.price
model = make_pipeline(column,forest)
model.fit(X_train,y_train)
model.score(X_train,y_train)

The model is seeming fit almost perfectly on the training data. Let's cross validate it on all 3 metrics.

In [ ]:
# Cross validation
X_valid = valid_set.drop('price',axis = 1)
y_valid = valid_set.price
cross_val_score(model,X_valid,y_valid,cv = 5).mean()

The cross validation score on valid set is also very good. Let's finally calculate the metrics on the test data.

In [ ]:
# Calculating metrics on test data
X_test = train_set.drop('price',axis = 1)
y_test = train_set.price
test_predict = model.predict(X_test)
scores = {
    "R^2 score" : r2_score(y_test,test_predict),
    "Mean Absolute Error" : mean_absolute_error(y_test,test_predict),
    "Mean Squared Error" : mean_squared_error(y_test,test_predict)
}
print(scores)

These metrics show that the model is fit very nicely, although the r2 score looks like it has overfitted. Let's now export the model and end the project.

In [ ]:
# Exporting the model
import joblib
joblib.dump(model,"final_model.joblib")